<a href="https://colab.research.google.com/github/AruneshDev/Analyzing-Historical-Stock-Revenue-Data-and-Building-a-Dashboard/blob/main/World_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit
!pip install streamlit ngrok


### Fetching a live stock data

In [ ]:
import yfinance as yf
import pandas as pd
import plotly.express as px

# Define top 20 tech stock tickers
top_tech_tickers = [
    "AAPL", "MSFT", "GOOGL", "AMZN", "TSLA", "NVDA", "META", "ADBE", "NFLX", "CRM",
    "AMD", "INTC", "ORCL", "IBM", "CSCO", "UBER", "PYPL", "SHOP", "SQ", "SNOW"
]
# Function to get live stock data and market cap
def get_stock_data(tickers):
    data = []
    for ticker in tickers:
        try:
            stock = yf.Ticker(ticker)
            info = stock.info
            hist = stock.history(period="1d", interval="5m")  # 5-minute interval
            latest = hist.iloc[-1]  # Get the most recent data

            data.append({
                "Ticker": ticker,
                "Price": latest["Close"],
                "Change (%)": ((latest["Close"] - latest["Open"]) / latest["Open"]) * 100,
                "Market Cap": info.get("marketCap", 0)  # Get market cap (0 if unavailable)
            })
        except Exception as e:
            print(f"Error fetching data for {ticker}: {e}")
    return pd.DataFrame(data)
# Fetch stock data
df = get_stock_data(top_tech_tickers)

# Convert Market Cap to Billions
df["Market Cap (B)"] = df["Market Cap"] / 1e9  # Convert to billions

# Display DataFrame
print(df)

### Stock Real-Time Heatmap

In [21]:
fig = px.treemap(df,
                 path=["Ticker"],
                 values="Market Cap",  # ✅ Fix: Use "Market Cap"
                 color="Change (%)",
                 color_continuous_scale=["red", "white", "green"],
                 title="Live Tech Stock Heatmap (Sized by Market Cap)")

fig.show()

### Streamlit Dashboard

### Investment Calculator

In [ ]:
import streamlit as st
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt

# Define stock options
stock_options = {
    "Apple (AAPL)": "AAPL",
    "Microsoft (MSFT)": "MSFT",
    "Google (GOOGL)": "GOOGL",
    "Amazon (AMZN)": "AMZN",
    "Tesla (TSLA)": "TSLA",
    "Nvidia (NVDA)": "NVDA",
    "Meta (META)": "META",
    "Adobe (ADBE)": "ADBE",
    "Netflix (NFLX)": "NFLX",
    "Salesforce (CRM)": "CRM"
}

# Function to calculate compound returns
def calculate_returns(amount, years, annual_return):
    return amount * (1 + annual_return / 100) ** years

# Function to fetch CAGR safely with retry mechanism
@st.cache_data
def get_cagr(stock_ticker):
    try:
        stock = yf.Ticker(stock_ticker)
        hist = stock.history(period="5y")

        if hist.empty:
            return 10, f"⚠️ No data available for {stock_ticker}, using default 10% CAGR."

        start_price = hist["Close"].iloc[0]
        end_price = hist["Close"].iloc[-1]
        cagr = ((end_price / start_price) ** (1 / 5) - 1) * 100
        return round(cagr, 2), None  # No warning message
    except Exception as e:
        return 10, f"⚠️ Error fetching data: {e}. Using default 10% CAGR."

# Streamlit UI (Main Execution)
def main():
    st.sidebar.header("📈 Stock Investment Calculator")

    # Dropdown for stock selection
    selected_stock = st.sidebar.selectbox("📊 Select a Stock", list(stock_options.keys()))
    selected_ticker = stock_options[selected_stock]

    # Fetch stock CAGR safely
    expected_return, warning_message = get_cagr(selected_ticker)

    # Display warning if needed
    if warning_message:
        st.sidebar.warning(warning_message)

    # User inputs
    amount = st.sidebar.number_input("💰 Investment Amount ($)", min_value=100, value=1000, step=100)
    years = st.sidebar.slider("⏳ Investment Duration (Years)", 1, 30, value=5)

    # Calculate projected return
    returns = calculate_returns(amount, years, expected_return)

    # Display estimated returns
    st.sidebar.subheader("💵 Estimated Returns")
    st.sidebar.write(f"📊 **Stock:** {selected_stock} ({selected_ticker})")
    st.sidebar.write(f"📈 **Historical Growth Rate:** {expected_return}% CAGR")
    st.sidebar.write(f"💰 **Investment:** ${amount:,} for {years} years")
    st.sidebar.write(f"💸 **Projected Return:** **${returns:,.2f}**")

    # Investment Growth Chart
    x = np.arange(0, years + 1)
    y = [calculate_returns(amount, i, expected_return) for i in x]

    fig, ax = plt.subplots(figsize=(6, 4))  # Increased figure size
    ax.plot(x, y, marker="o", linestyle="-", color="green", label="Investment Growth")
    ax.set_title(f"Investment Growth Over Time for {selected_ticker}")
    ax.set_xlabel("Years")
    ax.set_ylabel("Total Value ($)")
    ax.legend()
    ax.grid(True)
    ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'${x:,.0f}'))  # Format Y-axis as currency

    # Ensure Streamlit runs in context
    st.sidebar.pyplot(fig)

# Run Streamlit app
if __name__ == "__main__":
    main()
